如何精确地记录一篇文章的阅读量是一个比较复杂的问题，不过对于我们的博客来说，没有必要记录的那么精确。因此我们使用一种简单但有效的方式来记录博客文章的阅读量：文章每被浏览一次，则其阅读量 +1，即所谓的文章页面 **PV（Page View）数**。虽然简单粗暴，但却高效实用。

# 增加新字段

In [ ]:
# blog/models.py

class Post(models.Model):
    # ... 其它已有字段
    # 新增 views 字段记录阅读量
    views = models.PositiveIntegerField(default=0)

注意 views 字段的类型为 PositiveIntegerField，该类型的值只允许为正整数或 0，因为阅读量不可能为负值。初始化时 views 的值为 0。

# 增加模型方法

一旦用户访问了某篇文章，这时就应该将 views 的值 +1，这个过程最好由 Post 模型自己来完成，因此再给模型添加一个自定义的方法：

In [ ]:
# blog/models.py

class Post(models.Model):
    # ... 其它已有字段
    # 新增 views 字段记录阅读量
    views = models.PositiveIntegerField(default=0)

    # ... 其它已有的模型方法

    def increase_views(self):
        self.views += 1
        self.save(update_fields=['views'])

调用 save 方法将更改后的值保存到数据库。注意这里使用了 update_fields 参数来告诉 Django 只更新数据库中 views 字段的值，以提高效率。

# 迁移数据库

# 修改视图函数

In [ ]:
# blog/views.py

def detail(request, pk):
    post = get_object_or_404(Post, pk=pk)

    # 阅读量 +1
    post.increase_views()

    post.body = markdown.markdown(post.body,
                                  extensions=[
                                      'markdown.extensions.extra',
                                      'markdown.extensions.codehilite',
                                      'markdown.extensions.toc',
                                  ])
    form = CommentForm()
    comment_list = post.comment_set.all()
    context = {'post': post,
               'form': form,
               'comment_list': comment_list
               }
    return render(request, 'blog/detail.html', context=context)

# 在模板中显示阅读量

在模板中显示阅读量和显示其它字段一样，只需要使用模板变量即可。即模板适当的地方使用 {{ post.views }} 模板变量。这里我们分别修改两个地方，分别是 index.html 和 detail.html。

```HTML
templates/blog/index.html

<div class="entry-meta">
  ...
  <span class="views-count"><a href="{{ post.get_absolute_url }}">{{ post.views }} 阅读</a></span>
</div>
```

```HTML
templates/blog/detail.html

<div class="entry-meta">
  ...
  <span class="views-count"><a href="#">{{ post.views }} 阅读</a></span>
</div>
```